# LAB 3

Networks and Systems Security
Week 03
Authentication and Access Control



## Aims of the Seminar

Welcome to the Authentication and Access Control workshop. In this workshop, we'll
dive into the essential principles of modern authentication and access control. We'll
explore why simple password checks aren't enough and build a robust system from
the ground up using Python.

Workshop Outline:
1. Password Strength Analysis
2. Password Hashing Methods
3. Salt and Pepper Implementation
4. TOTP (2FA) Implementation
5. Brute Force Attack Simulation
6. Complete Authentication System

### Setting up

In [1]:
pip install bcrypt pyotp qrcode pillow

Note: you may need to restart the kernel to use updated packages.


### Section 1: Password Strength Analysis

In [24]:
import math
import string

#function that give a password a strength score
def password_strength(password):
    score = 0
    pool_size = 0
    length = len(password)

    # length check
    if length >= 8:
        score += 1
    if length >= 12:
        score += 1

    # character variety
    if any(c.islower() for c in password):
        score += 1
        pool_size += 26
    if any(c.isupper() for c in password):
        score += 1
        pool_size += 26
    if any(c.isdigit() for c in password):
        score += 1
        pool_size += 10
    if any(1 for c in password if c in string.punctuation):
        score += 1
        pool_size += 32  

    #entropy 
    if pool_size > 0:
        entropy = length * math.log2(pool_size)
        # print(f"Entropy: {entropy}")
        if entropy >= 25:
            score += 1
        if entropy >= 50:
            score += 1
        if entropy >= 75:
            score += 1
        if entropy >= 100:
            score += 1
       

    #common password check
    common_passwords = {"password", "123456", "qwerty", "abc123"}
    if password.lower() in common_passwords:
        score = 0  

    return score


### testing ###
# passwords = [
#     "password",
#     "a", 
#     "A",
#     "Aa",
#     "Aa123",
#     "Aa123!",
#     "aaaaaaaaaaaaaaaaaaaaa",
#     "aAaAaAaAaAaaaAaAaAa",
#     "AaBbCc12345678£$%!",
#     "Pass123",
#     "MyP@ssw0rd"
# ]
# for pwd in passwords:
#     print(f"Password: {pwd}, Strength Score: {password_strength(pwd)}")
##################

password = input("Enter a password: ")
print(f"Password strength score (1-10): {password_strength(password)}")

Password strength score (1-10): 8


### Section 2: Don't Store Passwords, Store Hashes!


In [25]:
import hashlib
import bcrypt

password = "MyPassw0rd123!"

##hashing passwords##

#using md5
print("MD5:     ", hashlib.md5(password.encode()).hexdigest())

#using sha-256
print("SHA-256: ", hashlib.sha256(password.encode()).hexdigest())

#using bcrypt
salt = bcrypt.gensalt()
hashed = bcrypt.hashpw(password.encode(), salt)
print("Bcrypt:  ", hashed)


MD5:      e60abbe4f45ad3999e51f09785a30b4f
SHA-256:  02891c9977283692a20c6cde56bf9ba8fa2e3fe023b96e6dee32b89467739cac
Bcrypt:   b'$2b$12$EXYrn75ul/KyLyRnMRqlW.XNB2IZGURargdmULrjVtM/5o22g.Q4i'


### Section 3: Adding Salt and Pepper

In [26]:
import hashlib
import bcrypt

password = "MyPassw0rd123!"

#hashing with no salt produces same hash for same password
print("first    SHA-256 (no salt):              ", hashlib.sha256(password.encode()).hexdigest())
print("second   SHA-256 (no salt):              ", hashlib.sha256(password.encode()).hexdigest())

#hashing with salt produces different hash for same password
print("first    SHA-256 (with salt):            ", hashlib.sha256((password + bcrypt.gensalt().decode()).encode()).hexdigest())
print("second   SHA-256 (with salt):            ", hashlib.sha256((password + bcrypt.gensalt().decode()).encode()).hexdigest())

#adding pepper
pepper = "PEPPER!"
print("first    SHA-256 (with salt and pepper): ", hashlib.sha256((password + bcrypt.gensalt().decode() + pepper).encode()).hexdigest())
print("second   SHA-256 (with salt and pepper): ", hashlib.sha256((password + bcrypt.gensalt().decode() + pepper).encode()).hexdigest())


first    SHA-256 (no salt):               02891c9977283692a20c6cde56bf9ba8fa2e3fe023b96e6dee32b89467739cac
second   SHA-256 (no salt):               02891c9977283692a20c6cde56bf9ba8fa2e3fe023b96e6dee32b89467739cac
first    SHA-256 (with salt):             d5430a15828012d80758c3f953d9a9c0bf95b2b91d2d7e90c4c058f67f8d5e2e
second   SHA-256 (with salt):             985212496cfb2bbbd0ba198c9759c44bf055b29de36ed021d15bbcf96f3dbedf
first    SHA-256 (with salt and pepper):  f3096b077c3548bd08cbb5fef57aa4286cfe0fbf85139205e665549e2028f19a
second   SHA-256 (with salt and pepper):  3b8e9fb756afd19b9d001231f249403da30d30e795c1a7f431785916e62c68cf


### Section 4: Implementing Two-Factor Authentication (2FA)

In [27]:
import pyotp
import time

#one time password generation

totp = pyotp.TOTP(pyotp.random_base32())  
print("Current OTP:", totp.now())

if totp.verify(input("Enter OTP to verify: ")):
    print("OTP is valid!")
else:
    print("OTP is invalid!")

time.sleep(10)



Current OTP: 414564
OTP is valid!


### Section 5: Simulating a Brute-Force Attack

In [31]:
import hashlib
import time

#dictionary attack function

def dictionary_attack(target_hash, hash_type):
    start = time.time()
    with open("100k-most-used-passwords.txt", "r") as f:
        common_passwords = f.read().splitlines()
    found = False
    for pwd in common_passwords:
        if hash_type == "md5":
            hashed_pwd = hashlib.md5(pwd.encode()).hexdigest()
        elif hash_type == "sha256":
            hashed_pwd = hashlib.sha256(pwd.encode()).hexdigest()
        else:
            print("Unsupported hash type")
            return

        if hashed_pwd == target_hash:
            found = True
            print(f"Password found using {hash_type}: {pwd}")
            print(f"Time taken: {time.time() - start} seconds\n")
            return
    if not found:
        print(f"Password not found in using {hash_type}.")
        print(f"Time taken: {time.time() - start} seconds\n")
          
        
## testing ###
# passwords = [
#     "password",
#     "a", 
#     "A",
#     "Aa",
#     "Aa123",
#     "Aa123!",
#     "aaaaaaaaaaaaaaaaaaaaa",
#     "aAaAaAaAaAaaaAaAaAa",
#     "AaBbCc12345678£$%!",
#     "Pass123",
#     "MyP@ssw0rd"
# ]
# for pwd in passwords:
#     dictionary_attack(hashlib.md5(pwd.encode()).hexdigest(), "md5")
#     dictionary_attack(hashlib.sha256(pwd.encode()).hexdigest(), "sha256")
#################


password = input("Enter a password: ")

dictionary_attack(hashlib.md5(password.encode()).hexdigest(), "md5")
dictionary_attack(hashlib.sha256(password.encode()).hexdigest(), "sha256")

Password found using md5: cheese
Time taken: 0.0070493221282958984 seconds

Password found using sha256: cheese
Time taken: 0.005006551742553711 seconds

